In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00001-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00002-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/README.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model.bin.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/special_tokens_map.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/.gitattributes
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.model
/kaggle/input/llama-2/pyt

In [3]:
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
#!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.0
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [4]:
from trl import SFTTrainer

2024-05-03 12:44:16.780835: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 12:44:16.780932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 12:44:16.913212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers

model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

# Dataset

In [7]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [8]:
dataset_train = dataset["train"].select(range(500))
dataset_validation = dataset["validation"].select(range(100))
dataset_test = dataset["test"].select(range(100))

print(dataset_train)
print(dataset_validation)
print(dataset_test)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 500
})
Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 100
})
Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 100
})


In [9]:
# Convert the dataset to a pandas DataFrame
df = dataset_train.to_pandas()

# View the first 5 rows of the DataFrame
first_five_rows = df.head()
print(first_five_rows)

                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a  


In [10]:
first_five_rows['highlights'][4]

"NEW: NFL chief, Atlanta Falcons owner critical of Michael Vick's conduct .\nNFL suspends Falcons quarterback indefinitely without pay .\nVick admits funding dogfighting operation but says he did not gamble .\nVick due in federal court Monday; future in NFL remains uncertain ."

# Zero Shot Prompt Engineering

In [ ]:
df = dataset_test.to_pandas()

In [ ]:
template ="""
<s>[INST] <<SYS>>
Summarize the following article.

Here is the article:
{article}

[/INST]

"""

syntheses_with_llama = []

for idx, row in df.iterrows():
    article = row['article']

  # Construct prompt with context
    prompt = template.format(article=article)
    #print(prompt)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate response
    output = model.generate(input_ids=inputs["input_ids"], max_new_tokens=100)

    # Decode the response
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    #Finding Extracted Summary
    answer_start = generated_text.find("[/INST]") + len("[/INST]")  # Find the end of </INST> tag
    model_summary = generated_text[answer_start:]  # Extract everything after that position
    syntheses_with_llama.append(model_summary)
    #print(answer)
    print(idx+1)
    
    #Comparing the answer with the base answer
    dash_line = '-'.join('' for x in range(100)) 
#    summary = first_five_rows.loc[idx, 'highlights']
#     print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
#     print(dash_line)
#     print(f'MODEL SUMMARY - ZERO SHOT:\n{model_summary}')
#     print(dash_line)
    
#Appending it to main file    
df['Generated_BY_LLAMA2'] = syntheses_with_llama

In [ ]:
df.head()

In [ ]:
from IPython.display import FileLink
Zero_Shot_LLama2 = df.rename(columns={'article': 'article', 'highlights': 'highlights', 'id': 'id', 'Generated_BY_LLAMA2': 'Generated_BY_LLAMA2'})
Zero_Shot_LLama2.to_csv('Zero_Shot_LLama2.csv', index=False)
FileLink('Zero_Shot_LLama2.csv')

# Zero Shot Test

In [45]:
import pandas as pd

file_path_1 = "/kaggle/input/llama2-cnn-daily/Zero_Shot_LLama2.csv"
Fine_Tuned_1 = pd.read_csv(file_path_1)

Fine_Tuned_1.head()

,article,highlights,id,Generated_BY_LLAMA2
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01,\n\nThe Palestinian Authority officially becam...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,\n\nA stray dog named Theia in Washington Stat...
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69,\n\nThe article provides an in-depth look at t...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,"\n\nAccording to the article, five Americans w..."
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,\n\nA Duke University student has been caught ...


### BLUE, ROUGE 1, ROUGE 2 and ROUGE L score

In [ ]:
!pip install rouge_score
!pip install scikit-learn

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
     

In [ ]:
def calculate_metrics(reference, candidate):
    # Tokenize reference and candidate
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=SmoothingFunction().method7)
    
    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)

    return bleu_score, scores

In [46]:
# Iterate through rows of the DataFrame and calculate scores
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for index, row in Fine_Tuned_1.iterrows():
    reference = row['highlights']
    candidate = row['Generated_BY_LLAMA2']
    
    bleu_score, rouge_scores = calculate_metrics(reference, candidate)
    
    bleu_scores.append(bleu_score)
    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

# Calculate average scores
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)

# Print average scores
print(f"Average BLEU Score: {avg_bleu_score}")
print(f"Average ROUGE-1 Score: {avg_rouge1_score}")
print(f"Average ROUGE-2 Score: {avg_rouge2_score}")
print(f"Average ROUGE-L Score: {avg_rougeL_score}")

Average BLEU Score: 0.224262075366063
Average ROUGE-1 Score: 0.32436095300847845
Average ROUGE-2 Score: 0.12343835855615584
Average ROUGE-L Score: 0.22595193530989247


### Bert Score

In [ ]:
!pip install bert_score

In [47]:
from bert_score import score
import pandas as pd

# Initialize lists to store scores
avg_precision_list = []
avg_recall_list = []
avg_f1_list = []

# Iterate through rows of the DataFrame and calculate scores
for index, row in Fine_Tuned_1.iterrows():
    reference = [row['highlights']]
    candidate = [row['Generated_BY_LLAMA2']]
    
    # Calculate BERTScore
    P, R, F1 = score(candidate, reference, lang="en")
    
    avg_precision = P.mean().item()
    avg_recall = R.mean().item()
    avg_f1 = F1.mean().item()
    
    avg_precision_list.append(avg_precision)
    avg_recall_list.append(avg_recall)
    avg_f1_list.append(avg_f1)

# Calculate average precision, recall, and F1 score
avg_precision = sum(avg_precision_list) / len(avg_precision_list)
avg_recall = sum(avg_recall_list) / len(avg_recall_list)
avg_f1 = sum(avg_f1_list) / len(avg_f1_list)

# Print average precision, recall, and F1 score
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1 Score: {avg_f1}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Average Precision: 0.8597955405712128
Average Recall: 0.8790707474946976
Average F1 Score: 0.869227740764618


# Fine Tuning

In [11]:
dataset_train = dataset_train.to_pandas()
dataset_validation = dataset_validation.to_pandas()

In [12]:
import pandas as pd
from datasets import Dataset


prompts = []
for idx, row in dataset_train.iterrows():
    article = row['article']
    summary = row['highlights']
    
    SYSTEM_PROMPT = """Summarize the following article."""
    
    prompt = f"<s>[INST]\n<<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n{article}\n[/INST]\n{summary}\n</s>"
    prompts.append(prompt)

dataset = Dataset.from_pandas(pd.DataFrame({'text': prompts}))
dataset


Dataset({
    features: ['text'],
    num_rows: 500
})

In [13]:
prompts = []
for idx, row in dataset_validation.iterrows():
    article = row['article']
    summary = row['highlights']
    
    SYSTEM_PROMPT = """Summarize the following article."""
    
    prompt = f"<s>[INST]\n<<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n{article}\n[/INST]\n{summary}\n</s>"
    prompts.append(prompt)

eval_dataset = Dataset.from_pandas(pd.DataFrame({'text': prompts}))
eval_dataset


Dataset({
    features: ['text'],
    num_rows: 100
})

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [16]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    "Fine_Tuned_LLama2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-5,
    fp16=True,
    weight_decay=0.01,
    max_grad_norm=0.3,
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    lr_scheduler_type="cosine",
    seed=42,
    push_to_hub = True,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=100,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
31,2.156200,2.268043
62,1.602300,1.826826
93,1.534600,1.664935
124,1.389100,1.639921
155,1.409300,1.636870


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /kaggle/input/llama-2/pytorch/7b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /kaggle/input/llama-2/pytorch/7b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: t

TrainOutput(global_step=155, training_loss=1.8123304297847131, metrics={'train_runtime': 2375.7043, 'train_samples_per_second': 1.052, 'train_steps_per_second': 0.065, 'total_flos': 9856691798016000.0, 'train_loss': 1.8123304297847131, 'epoch': 4.96})

In [20]:
lora = trainer.model.save_pretrained('lora_adapter')

In [24]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# import transformers
# from peft import PeftModel

# # base_model_name = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
# # adapter_model_name = "/kaggle/working/lora_adapter"

# # bnb_config = BitsAndBytesConfig(
# #     load_in_4bit=True,
# #     bnb_4bit_use_double_quant=True,
# #     bnb_4bit_quant_type="nf4",
# #     bnb_4bit_compute_dtype=torch.bfloat16
# # )

# # model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")


# # model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=bnb_config, device_map='auto', torch_dtype=torch.float16)
# # model = PeftModel.from_pretrained(model, adapter_model_name, quantization_config=bnb_config, device_map='auto', torch_dtype=torch.float16)

# # Merge the adapters into the base model so you can use the model like a normal transformers model
# model = model.merge_and_unload(model, lora)
# model.save_pretrained('final_model')

# tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Prediction

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers

model_id_1 = '/kaggle/working/Fine_Tuned_LLama2'


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

fine_tuned_llama2 = AutoModelForCausalLM.from_pretrained(model_id_1, quantization_config=bnb_config, device_map="auto")
tokenizer_1 = AutoTokenizer.from_pretrained(model_id_1)
tokenizer_1.pad_token = tokenizer_1.eos_token
tokenizer_1.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

In [25]:
dataset_test = dataset_test.to_pandas()

In [26]:
len(dataset_test)

100

In [27]:
template ="""
<s>[INST] <<SYS>>
Summarize the following article.

Here is the article:
{article}

[/INST]

"""

syntheses_with_llama = []

for idx, row in dataset_test.iterrows():
    article = row['article']

  # Construct prompt with context
    prompt = template.format(article=article)
    #print(prompt)

    # Tokenize the prompt
    inputs = tokenizer_1(prompt, return_tensors="pt")

    # Generate response
    output = fine_tuned_llama2.generate(input_ids=inputs["input_ids"], max_new_tokens=100)

    # Decode the response
    generated_text = tokenizer_1.decode(output[0], skip_special_tokens=True)
    
    #Finding Extracted Summary
    answer_start = generated_text.find("[/INST]") + len("[/INST]")  # Find the end of </INST> tag
    model_summary = generated_text[answer_start:]  # Extract everything after that position
    syntheses_with_llama.append(model_summary)
    #print(answer)
    print(idx+1)
    
    #Comparing the answer with the base answer
    dash_line = '-'.join('' for x in range(100)) 
#    summary = first_five_rows.loc[idx, 'highlights']
#     print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
#     print(dash_line)
#     print(f'MODEL SUMMARY - ZERO SHOT:\n{model_summary}')
#     print(dash_line)
    
#Appending it to main file    
dataset_test['Generated_BY_LLAMA2'] = syntheses_with_llama

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [28]:
dataset_test.head()

,article,highlights,id,Generated_BY_LLAMA2
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01,\n\nThe Palestinian Authority officially becam...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,\n\nA stray dog in Washington State named Thei...
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69,\n\nThe article provides information about Ira...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,\n\nFive Americans who were exposed to Ebola i...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,\n\nA Duke University student has admitted to ...


In [29]:
from IPython.display import FileLink
Fine_Tune_LLama2 = dataset_test.rename(columns={'article': 'article', 'highlights': 'highlights', 'id': 'id', 'Generated_BY_LLAMA2': 'Generated_BY_LLAMA2'})
Fine_Tune_LLama2.to_csv('Fine_Tune_LLama2.csv', index=False)
FileLink('Fine_Tune_LLama2.csv')

/kaggle/working/Fine_Tune_LLama2.csv

# Fine Tune Test

### BLUE, ROUGE 1, ROUGE 2 and ROUGE L score

In [30]:
!pip install rouge_score
!pip install scikit-learn

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=485a883a1539b87dd427062bae336e49540d631d523d417e7c3425d0c6170eb7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [39]:
def calculate_metrics(reference, candidate):
    # Tokenize reference and candidate
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=SmoothingFunction().method7)
    
    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)

    return bleu_score, scores

In [40]:
# Iterate through rows of the DataFrame and calculate scores
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for index, row in Test_Fine_Tune.iterrows():
    reference = row['highlights']
    candidate = row['Generated_BY_LLAMA2']
    
    bleu_score, rouge_scores = calculate_metrics(reference, candidate)
    
    bleu_scores.append(bleu_score)
    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

# Calculate average scores
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)

# Print average scores
print(f"Average BLEU Score: {avg_bleu_score}")
print(f"Average ROUGE-1 Score: {avg_rouge1_score}")
print(f"Average ROUGE-2 Score: {avg_rouge2_score}")
print(f"Average ROUGE-L Score: {avg_rougeL_score}")

Average BLEU Score: 0.21364825828199133
Average ROUGE-1 Score: 0.3238532843724999
Average ROUGE-2 Score: 0.1260455259765372
Average ROUGE-L Score: 0.22647781346443493


In [36]:
# Test_Fine_Tune = Fine_Tune_LLama2[['highlights', 'Generated_BY_LLAMA2']]

In [41]:
# Test_Fine_Tune.columns

### BERT_Score

In [42]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00


In [44]:
from bert_score import score
import pandas as pd

# Initialize lists to store scores
avg_precision_list = []
avg_recall_list = []
avg_f1_list = []

# Iterate through rows of the DataFrame and calculate scores
for index, row in Test_Fine_Tune.iterrows():
    reference = [row['highlights']]
    candidate = [row['Generated_BY_LLAMA2']]
    
    # Calculate BERTScore
    P, R, F1 = score(candidate, reference, lang="en")
    
    avg_precision = P.mean().item()
    avg_recall = R.mean().item()
    avg_f1 = F1.mean().item()
    
    avg_precision_list.append(avg_precision)
    avg_recall_list.append(avg_recall)
    avg_f1_list.append(avg_f1)

# Calculate average precision, recall, and F1 score
avg_precision = sum(avg_precision_list) / len(avg_precision_list)
avg_recall = sum(avg_recall_list) / len(avg_recall_list)
avg_f1 = sum(avg_f1_list) / len(avg_f1_list)

# Print average precision, recall, and F1 score
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1 Score: {avg_f1}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Average Precision: 0.8606414878368378
Average Recall: 0.8794602960348129
Average F1 Score: 0.8698611944913864
